# Fig. 1b materials

In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import sys
sys.path.append("../") # Set parent directory to sys.path
sys.dont_write_bytecode = True
%load_ext autoreload
%autoreload 2
import src.utils as utils
import src.map_vis as mv

palette0 = sns.color_palette(['#E69F00', '#56B4E9', '#009E73', '#F0E442', '#0072B2', '#D55E00', '#CC79A7', '#000000'])
palette2 = sns.color_palette(["#D81B60", "#1E88E5", "#FFC107", "#004D40"])
palette = palette2
display(palette)
sns.set_palette(palette)

sns.set_theme(context='poster', style='ticks', palette=palette, font_scale=1.0)

## IMU data

In [ ]:
sns.set_theme(context='poster', style='ticks', palette=palette, font_scale=1.5)
# Change the path appropriately
# save_dir = "../path_to_save_dir"
save_dir = "C:/Users/ryoma/D/writing/00-first/005_Otsuka_202x_MEE_umineko_playback/otsuka-umineko-playback/illustration/materials"
target_path = "../data/extracted-imu-data/*/*.csv"
path_list = sorted(glob.glob(target_path))
print(f"Number of csv files: {len(path_list)}")

### Accleration

In [ ]:
# data
df = pd.read_csv(path_list[6])

# plot and customize
fig, ax = plt.subplots(1, 1, figsize=(18*0.7, 5*0.7))
ax = utils.plot_imu_data_v5(
    df, ax, session_name=None, data_type="acc", filter_type=None, 
    sampling_rate=25, plot_before_sec=12, plot_after_sec=12+5, title=None,
    x_interval_sec=4
)
ax.set_xlabel("Time (s)", labelpad=12)
ax.set_ylabel("Acc (g)", labelpad=12)
ax.set_yticks(np.arange(-8, 8, 4))
ax.set_ylim(-5.5, 5.5)
ax.grid(axis='both', which='major', alpha=0.2)
ax.grid(axis='both', which='minor', alpha=0.1)
ax.legend(ncol=3, loc='lower right', columnspacing=0.9, handlelength=0.7)

# save
filename = "fig_1b_acc"
# fig.savefig(f"{save_dir}/{filename}.svg", bbox_inches="tight", pad_inches=0.25, transparent=False)
# fig.savefig(f"{save_dir}/{filename}.png", bbox_inches="tight", pad_inches=0.25, transparent=False)
plt.show()
plt.close()

### Gyro (low pass filter)

In [ ]:
# data
# df = pd.read_csv(path_list[6])
df = pd.read_csv(path_list[7])

# plot and customize
fig, ax = plt.subplots(1, 1, figsize=(18*0.7, 5*0.7))
ax = utils.plot_imu_data_v5(
    df, ax, session_name=None, data_type="gyro", filter_type="butter", 
    sampling_rate=25, plot_before_sec=12, plot_after_sec=12+5, title=None,
    x_interval_sec=4
)
ax.set_xlabel("Time (s)", labelpad=12)
ax.set_ylabel("Gyro ($^\circ$/s)", labelpad=12)
ax.grid(axis='both', which='major', alpha=0.2)
ax.grid(axis='both', which='minor', alpha=0.1)
ax.legend(ncol=3, loc='lower right', columnspacing=0.9, handlelength=0.7)

# save
filename = "fig_1b_gyro"
# fig.savefig(f"{save_dir}/{filename}.svg", bbox_inches="tight", pad_inches=0.25, transparent=False)
# fig.savefig(f"{save_dir}/{filename}.png", bbox_inches="tight", pad_inches=0.25, transparent=False)
plt.show()
plt.close()


## GPS data

### Real trajectory | umineko-2024 GPS data

In [ ]:
# Kabushima shrine, Hachinohe city, Aomori, Japan
kabushima_shrine = [40.538570, 141.557596] 
# save_dir = "../output/figure/"
save_dir = "C:/Users/ryoma/D/writing/00-first/005_Otsuka_202x_MEE_umineko_playback/otsuka-umineko-playback/illustration/materials"

In [ ]:
test_id_list = [
    "LBP00",
    "LBP01",
    "LBP03",
    "LBP05",
    "LBP06",
    "LBP07",
    "LBP08",
    "LBP09",
]

In [ ]:
df_all = pd.DataFrame()
target_path = "../data/extracted-gps-data-fixed/*"
path_list = sorted(glob.glob(target_path))
for i, path in enumerate(path_list):
    # print("--------------------------------------------------")
    test_id = os.path.basename(path).replace(".csv", "")
    if test_id not in test_id_list:
        continue # skip
    else:
        print(f"test_id: {test_id}")
    # load csv data
    df = pd.read_csv(path)
    df.insert(0, "test_id", [test_id]*len(df))
    df_all = pd.concat([df_all, df], axis=0)
df_all = df_all[df_all['fix_type'] == 3]
df_all.reset_index(inplace=True)
df = df_all
# print(len(df))
print(f"Number of GPS data points: {len(df_all)}")

df = df[['test_id', 'latitude', 'longitude', 'camera_count', 'camera_recording', 'speaker_on']]
display(df.head(3))
display(df.tail(3))
lats = df['latitude'].values
lons = df['longitude'].values
camera_recordings = df['camera_recording'].values
speaker_ons = df['speaker_on'].values

# assign one color per individual bird
color_list = ['#E69F00', '#56B4E9', '#F0E442', '#009E73', '#0072B2', '#D55E00', '#CC79A7', '#999999', '#000000']
color_map = dict(zip(test_id_list, color_list))
print(color_map)

# make colors list
colors_per_bird = df['test_id'].map(color_map).tolist()
print(colors_per_bird[:3])
print(colors_per_bird[-3:])
colors_per_bird = np.where(camera_recordings > 0.0, "#D81B60", colors_per_bird) # camera recording -> red

In [ ]:
fig = mv.vis_trajectory_map(
    lats, 
    lons, 
    colors_per_bird, 
    base_loc=kabushima_shrine
)

In [ ]:
# fig.savefig(f"{save_dir}/trajectory_map_all_per_bird.svg", bbox_inches="tight", pad_inches=0.2, transparent=False)
# fig.savefig(f"{save_dir}/trajectory_map_all_per_bird.png", dpi=350, bbox_inches="tight", pad_inches=0.20, transparent=False)

In [ ]:
# camera on/off only version
colors_camera_on_off = np.where(camera_recordings > 0.0, "#D81B60", "#FFC107") # camera recording -> red
fig = mv.vis_trajectory_map(
    lats, 
    lons, 
    colors_camera_on_off, 
    base_loc=kabushima_shrine,
    ax3_lats_lons_colors=None,
)

### Simulated trajectory
The simulated data were used to adjust the plot.

In [ ]:
# Kabushima shrine, Hachinohe city, Aomori, Japan
kabushima_shrine = [40.538570, 141.557596] 
N = 500

#### Random Walk

In [ ]:
random_lats, random_lons = mv.generate_random_trajectory(
    random_type="random-walk", 
    start_lat=40.70, 
    start_lon=141.70, 
    step_degree=0.005, 
    random_seed=3,
    n_steps=N
)
colors = ["#1E88E5"]*len(random_lats)
fig = mv.vis_trajectory_map(
    random_lats, 
    random_lons, 
    colors, 
    kabushima_shrine, 
    plot_line=True
)

#### Levy Flights

In [ ]:
levy_lats, levy_lons = mv.generate_random_trajectory(
    random_type="levy-flights",
    start_lat=40.70, 
    start_lon=141.70, 
    step_degree=0.005, 
    random_seed=3,
    n_steps=N
)
colors = ["#1E88E5"]*len(levy_lats)
fig = mv.vis_trajectory_map(
    levy_lats, 
    levy_lons, 
    colors, 
    kabushima_shrine, 
    plot_line=True
)